In [6]:
print("Hello from the GCP World")

Hello from the GCP World


So we want to train on the whole dataset I will use the minimum number of dimensions to make the vector for my microservice easier to work with SKlearn is not as well set up for large scale analysis as Pyspark, so we are seperating code and deleting variables to try and stop memory errors. 24 cpus is my limit on the free tier so the most powerful hardward I can use is 16cpu 60gb Ram

In [7]:
! PYTHONHASHSEED=0
import pandas as pd
import numpy as np

In [8]:
categoricalCols = [
    "wlan_tag_length",
    "wlan_country_info_fnm",
    "wlan_da",
    "wlan_sa",
    "wlan_bssid",
    "ip_src",
    "ip_dst"
]
   

In [9]:
numericCols=[
    "wlan_fixed_reason_code",
    "radiotap_channel_freq",
    "udp_length"
]

In [10]:
labelCols=[
    "StringLabel",
    "label"
]

In [11]:
Deauth=pd.read_parquet("gs://birkbeckwifibucket/parquet/1.Deauth", engine='pyarrow')
rDeauth=Deauth[categoricalCols+numericCols+labelCols]
del Deauth

In [12]:

Disas=pd.read_parquet("gs://birkbeckwifibucket/parquet/2.Disas", engine='pyarrow')
rDisas=Disas[categoricalCols+numericCols+labelCols]
del Disas

In [13]:
reAssoc=pd.read_parquet("gs://birkbeckwifibucket/parquet/3.(Re)Assoc", engine='pyarrow')
rreAssoc=reAssoc[categoricalCols+numericCols+labelCols]
del reAssoc

In [14]:
rogueAP=pd.read_parquet("gs://birkbeckwifibucket/parquet/4.Rogue_AP", engine='pyarrow')
rrogueAP=rogueAP[categoricalCols+numericCols+labelCols]
del rogueAP

In [15]:
krack=pd.read_parquet("gs://birkbeckwifibucket/parquet/5.Krack", engine='pyarrow')
rkrack=krack[categoricalCols+numericCols+labelCols]
del krack

In [16]:
Kr00k=pd.read_parquet("gs://birkbeckwifibucket/parquet/6.Kr00k", engine='pyarrow')
rKr00k=Kr00k[categoricalCols+numericCols+labelCols]
del Kr00k

In [17]:
ssh=pd.read_parquet("gs://birkbeckwifibucket/parquet/7.SSH", engine='pyarrow')
rSSH=ssh[categoricalCols+numericCols+labelCols]
del ssh

In [18]:
botnet=pd.read_parquet("gs://birkbeckwifibucket/parquet/8.Botnet", engine='pyarrow')
rBotnet=botnet[categoricalCols+numericCols+labelCols]
del botnet

In [19]:
malware=pd.read_parquet("gs://birkbeckwifibucket/parquet/9.Malware", engine='pyarrow')
rMalware=malware[categoricalCols+numericCols+labelCols]
del malware

In [20]:
sql=pd.read_parquet("gs://birkbeckwifibucket/parquet/10.SQL_Injection", engine='pyarrow')
rSQL=sql[categoricalCols+numericCols+labelCols]
del sql

In [21]:
ssdp=pd.read_parquet("gs://birkbeckwifibucket/parquet/11.SSDP", engine='pyarrow')
rSSDP=ssdp[categoricalCols+numericCols+labelCols]
del ssdp

In [22]:
evilTwin=pd.read_parquet("gs://birkbeckwifibucket/parquet/12.Evil_Twin", engine='pyarrow')
rEvilTwin=evilTwin[categoricalCols+numericCols+labelCols]
del evilTwin

In [23]:
ws=pd.read_parquet("gs://birkbeckwifibucket/parquet/13.Website_spoofing", engine='pyarrow')
rWS=ws[categoricalCols+numericCols+labelCols]
del ws

In [24]:
#we concat the reduced dataframes
frames = [rDeauth, rDisas,rreAssoc,rrogueAP,rkrack,rKr00k,rSSH,rBotnet,rMalware,rSQL,rSSDP,rEvilTwin,rWS]
dataset=pd.concat(frames)

In [25]:
#incase the next bit breaks
dataset.to_pickle("./pickledReducedColsDF.pkl")  

In [26]:
len(dataset.index)

27576415

Checking we have the whole dataset

In [27]:
dataset['label'].value_counts()

0    21660389
1     5916026
Name: label, dtype: int64

Seeing how many are attack packets

In [28]:
dataset['StringLabel'].value_counts()

Normal              21660389
SSDP                 5499851
Malware               131611
Disas                  75131
Botnet                 56891
Evil_Twin              49186
Deauth                 38942
Website_spoofing       25923
Krack                  16009
SSH                    11882
(Re)Assoc               5502
SQL_Injection           2629
RogueAP                 1310
Kr00k                   1138
NA                        21
Name: StringLabel, dtype: int64

And the types of attacks

In [29]:
 from sklearn.preprocessing import LabelEncoder, OneHotEncoder

In [30]:
y=dataset['label']
x=dataset[categoricalCols+numericCols]
x[categoricalCols].astype("category")

,wlan_tag_length,wlan_country_info_fnm,wlan_da,wlan_sa,wlan_bssid,ip_src,ip_dst
0,NA,NA,NA,NA,NA,NA,NA
1,NA,NA,NA,NA,NA,NA,NA
2,NA,NA,50:3e:aa:e4:01:93,0c:9d:92:54:fe:30,0c:9d:92:54:fe:34,151.101.17.140,192.168.2.73
3,NA,NA,50:3e:aa:e4:01:93,0c:9d:92:54:fe:30,0c:9d:92:54:fe:34,151.101.17.140,192.168.2.73
4,NA,NA,50:3e:aa:e4:01:93,0c:9d:92:54:fe:30,0c:9d:92:54:fe:34,151.101.17.140,192.168.2.73
...,...,...,...,...,...,...,...
99994,NA,NA,NA,NA,NA,NA,NA
99995,NA,NA,NA,NA,NA,NA,NA
99996,NA,NA,NA,NA,NA,NA,NA
99997,NA,NA,NA,NA,NA,NA,NA


In [31]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
categorical_transformer = OneHotEncoder(handle_unknown="ignore")
from sklearn.preprocessing import FunctionTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ("cat", categorical_transformer, categoricalCols)
    ]
)

In [32]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)


In [33]:
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier

In [34]:
RF = Pipeline(steps=[("preprocess", preprocessor), ("RFclassifier", RandomForestClassifier(max_depth=None, min_samples_split=2,max_features='log2',criterion='gini',n_estimators=100,random_state=0,n_jobs=-1))])

In [35]:
%%time
RF.fit(X_train, y_train)
print("The model score for a RandomForestClassifier: %.3f" % RF.score(X_test, y_test))
testPredictions=RF.predict(X_test)

The model score for a RandomForestClassifier: 0.993
CPU times: user 18h 17min 8s, sys: 1min 24s, total: 18h 18min 33s
Wall time: 1h 22min 15s


In [36]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import matthews_corrcoef
tn, fp, fn, tp = confusion_matrix(y_test, testPredictions).ravel()
precision,recall,fscore,support = precision_recall_fscore_support(y_test, testPredictions, average='binary')
MCC = matthews_corrcoef(y_test, testPredictions)
print("There are "+str(tn)+ " true negatives")
print("There are "+str(tp)+ " true positives")
print("There are "+str(fn)+ " false negatives")
print("There are "+str(fp)+ " false positives")
print("The precision is "+str(precision))
print("The recall is "+str(recall))
print("The fscore is "+str(fscore))
print("The Matthews correlation coefficient is "+str(MCC))

There are 4321570 true negatives
There are 1155444 true positives
There are 26916 false negatives
There are 11353 false positives
The precision is 0.9902699441290987
The recall is 0.9772353597888968
The fscore is 0.9837094753564789
The Matthews correlation coefficient is 0.9793362615765545


In [37]:
from google.cloud import storage
import joblib
storage_client = storage.Client()
model_filename = "fullDataSetModel.joblib"
with open(model_filename, "wb") as model_file:
    joblib.dump(RF, model_file)
    
bucket_name="birkbeckwifibucket"
bucket = storage_client.bucket(bucket_name)
blob = bucket.blob(model_filename)
blob.upload_from_filename(model_filename)

so the full dataset has excellent performance, better than my tests, however I would attribute a lot of this to the fact that the SSDP has by far the most attack packets and has a more or less one to one mapping with UDP length field. The code below saves the model so it can be used in my endpoint